In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [55]:
FILE_PATH = "dac/train.txt"
dataset_raw = sc.textFile(FILE_PATH)

In [56]:
def convert_value(index, value):
    if index < 14:
        return int(value) if value else None
    else:
        return value if value else None

def convert_line(line):
    return [convert_value(i, value) for i, value in enumerate(line.split("\t"))]

# data = lines.map(convert_line)

In [57]:
import time
import random
random.seed(time.time())

seed = random.random()

full_train, test = dataset_raw.randomSplit([10, 38], seed=seed)
train_5m, validation_2m, test_3m = full_train.randomSplit([5, 2, 3], seed=seed)

In [58]:
train_5m.take(1)

KeyboardInterrupt: 

In [50]:
# data.map(lambda row: row[0]).histogram([0, 1, 2])

In [53]:
# data.map(lambda row: row[2]).histogram(10)

In [54]:
# a = data.map(lambda row: row[2]).histogram([-10,0,5,10,50,100,200,500,1000,2000,10000,50000])

In [44]:
def show_histogram(col_num, data):
    bins, counts = data.map(lambda row: row[col_num]).histogram(10)
    total = sum(counts)
    print "column #%d: bins=%s, counts=%s (total=%d)" % (col_num, bins, counts, total)
    #sum the counts
    #max of the counts
    #if  > 25%
    
for i in range(1,14):
    show_histogram(i, data)

column #1: bins=[0.0, 55.6, 111.2, 166.8, 222.4, 278.0, 333.6, 389.2, 444.8, 500.40000000000003, 556], counts=[55236, 295, 34, 10, 5, 3, 2, 0, 1, 1] (total=55587)
column #2: bins=[-2.0, 1850.4, 3702.8, 5555.200000000001, 7407.6, 9260.0, 11112.400000000001, 12964.800000000001, 14817.2, 16669.600000000002, 18522], counts=[98530, 1397, 52, 12, 7, 0, 1, 0, 0, 1] (total=100000)
column #3: bins=[0.0, 6553.5, 13107.0, 19660.5, 26214.0, 32767.5, 39321.0, 45874.5, 52428.0, 58981.5, 65535], counts=[80863, 11, 1, 20, 0, 0, 0, 0, 1, 2] (total=80898)
column #4: bins=[0.0, 41.7, 83.4, 125.10000000000001, 166.8, 208.5, 250.20000000000002, 291.90000000000003, 333.6, 375.3, 417], counts=[79210, 1178, 44, 8, 12, 6, 5, 0, 0, 3] (total=80466)
column #5: bins=[0.0, 174112.8, 348225.6, 522338.39999999997, 696451.2, 870564.0, 1044676.7999999999, 1218789.5999999999, 1392902.4, 1567015.2, 1741128], counts=[93216, 1145, 491, 218, 88, 44, 19, 11, 5, 3] (total=95240)
column #6: bins=[0, 1629, 3258, 4887, 6516, 81